In [1]:
%load_ext autoreload
%autoreload 2
%aimport - pandas

In [4]:
import sys
sys.path.append('../../')
sys.path.append('../../../scripts')
# from create_cpt_sessions import create_pickle

In [5]:
marco_path = "/Users/fsp585/Desktop/GetherLabCode/FiberphotometryCode/src/11-12-2024_Marco/"
sam_path = "/Users/fsp585/Desktop/GetherLabCode/FiberphotometryCode/src/Baseline"

In [6]:
#cpt_sessions = create_cpt_sessions.create_pickle(baseline_path, None)
# cpt_sessions = create_pickle(sam_path, overwrite=True, first_n_dirs=3)
# cpt_sessions = create_pickle(marco_path, overwrite=True)

In [8]:
import os, sys, pkgutil, importlib, pickle

# 1) cd into project root (adjust this to your actual path)
os.chdir("/Users/fsp585/Desktop/GetherLabCode/FiberphotometryCode")
sys.path.insert(0, os.getcwd())

# 2) import your package
import fiberphotometry

# 3) alias every sub-package as a bare module
for _, name, _ in pkgutil.iter_modules(fiberphotometry.__path__):
    sys.modules[name] = importlib.import_module(f"fiberphotometry.{name}")

# 4) custom Unpickler fallback to fiberphotometry.<module>
class PrefixUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        try:
            return super().find_class(module, name)
        except ModuleNotFoundError:
            return super().find_class(f"fiberphotometry.{module}", name)

# 5) load your sessions
with open("/Users/fsp585/Desktop/GetherLabCode/FiberphotometryCode/src/11-12-2024_Marco", "rb") as f:
    sessions = PrefixUnpickler(f).load()

print(f"✅ Loaded {len(sessions)} sessions")


IsADirectoryError: [Errno 21] Is a directory: '/Users/fsp585/Desktop/GetherLabCode/FiberphotometryCode/src/11-12-2024_Marco'

In [ ]:
# sessions = [s for s in sessions if s.mouse_id == '45']

In [ ]:
for s in sessions:
    print(s.trial_dir, type(s.mouse_id))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import plotly.graph_objects as go

for s in sessions:
    phot_df = s.dfs.data['phot_470']
    for col in phot_df.columns:
        # only plot the 3‑length tuple columns
        if isinstance(col, tuple) and len(col) == 3:
            fig = go.Figure(
                data=go.Scatter(
                    x=phot_df.index,
                    y=phot_df[col],
                    mode='lines',
                    name=str(col)
                )
            )
            fig.update_layout(
                title=f"{s.trial_id}, mouse {s.mouse_id}, channel {col}",
                xaxis_title="Frame index",
                yaxis_title="z‑score",
                template="simple_white",
                width=800,
                height=400
            )
            fig.show()

In [ ]:
for s in sessions:
    try:
        print(s.signal_info['fixed_ratio_touch_fr1']['signal_matrix'].shape)
    except:
        continue

In [ ]:
from collections import defaultdict
from fiberphotometry.analysis.signal_processing import group_signals
from fiberphotometry.analysis.plotting import plot_matrix_ci

# 1) build the full groups dict
groups = group_signals(
    sessions,
    key_fn=lambda _, sig_key: (
        sig_key
    )
)

In [ ]:
import re
import bisect
from fiberphotometry.analysis.signal_processing import group_signals

# 1) define your monotonic bounds array:
#    e.g. bins = [0, 5, 10, 20] means:
#      bin0: X<5
#      bin1: 5<=X<10
#      bin2: 10<=X<20
bounds = [0, 5, 10, 20, 30]  

def cc_key_fn(session, sig_key):
    region, channel, ev = sig_key

    # 2) match only events that end with "_cc<digits>"
    m = re.search(r'_cc(\d+)$', ev)
    if not m:
        return None

    X = int(m.group(1))

    # 3) find bin index using bisect:
    #    bisect_right returns the insertion point; subtract 1 to get the bin
    bin_nr = bisect.bisect_right(bounds, X) - 1
    if bin_nr < 0 or bin_nr >= len(bounds)-1:
        # outside your defined bins
        return None

    # 4) cycle number mod 3
    cycle_nr = X % 3

    # 5) return just what you care about
    return (region, bin_nr, cycle_nr)

# now group
groups = group_signals(sessions, key_fn=cc_key_fn)
groups.pop(None, None)    # delete the None key if it exists

# inspect
for (region, bin_nr, cycle_nr), mat in groups.items():
    print(f"{region:>4} | bin {bin_nr:>2} | cycle {cycle_nr}: {mat.shape}")

In [ ]:
brain_reg_to_color = {'VS': 'purple',
                      'DMS': 'forestgreen',
                      'DLS': 'C0'}

In [ ]:
all_hit_num = []

for s in sessions:
    all_k = []
    for k in s.signal_info.keys():
        if k[-1].startswith('global') and 'cc' in k[-1]:
            all_k.append(k[-1])
    all_hit_num.append(int(all_k[-1].split('cc')[-1]))
    
print(sum(all_hit_num)/len(all_hit_num))

In [ ]:
# %% cell: Cycle-by-bin DA plot with autoscaled Y (full interval)
import re, bisect
import matplotlib.pyplot as plt
from fiberphotometry.analysis.signal_processing import group_signals, compute_mean_ci
from fiberphotometry.analysis.plotting      import get_time_axis

# 1) Define your bins and key_fn
bounds = [0, 20, 50, 100]
def cc_key_fn(session, sig_key):
    br, ch, ev = sig_key
    m = re.search(r'_cc(\d+)$', ev)
    if not m:
        return None
    X = int(m.group(1))
    bin_nr = bisect.bisect_right(bounds, X) - 1
    if bin_nr < 0 or bin_nr >= len(bounds)-1:
        return None
    cycle_nr = X % 3
    return (br, bin_nr, cycle_nr)

groups = group_signals(sessions, key_fn=cc_key_fn)
groups.pop(None, None)

# 2) Unique regions & bins
brain_regions = sorted({ br for (br, _, _) in groups })
bin_nrs       = list(range(len(bounds)-1))
cycle_nrs     = [1, 2]

# 3) Grid setup with independent Y axes
n_rows = len(brain_regions)
n_cols = len(bin_nrs)
fig, axes = plt.subplots(
    n_rows, n_cols,
    figsize=(4*n_cols, 3*n_rows),
    sharex=True,
    sharey=False    # allow each subplot to autoscale Y
)

xs = get_time_axis()
linestyles = ['-', '--', ':']

# 4) Plot mean±CI and autoscale Y per subplot
for i, br in enumerate(brain_regions):
    for j, b in enumerate(bin_nrs):
        ax = axes[i, j] if n_rows > 1 else axes[j]
        base_color = brain_reg_to_color.get(br, 'C0')

        for c, ls in zip(cycle_nrs, linestyles):
            key = (br, b, c)
            mat = groups.get(key)
            if mat is None:
                continue

            ys, lo, hi = compute_mean_ci(mat, alpha=0.5)

            ax.plot(xs, ys,
                    color=base_color,
                    linestyle=ls,
                    label=f'cycle {3 if c==0 else c}')
            # ax.fill_between(xs, lo, hi,
            #                 color=base_color, alpha=0.2)

        # Autoscale this subplot's Y axis
        ax.relim()
        ax.autoscale(axis='y')

        # Titles & labels
        lo_bound, hi_bound = bounds[b], bounds[b+1]
        if i == 0:
            ax.set_title(f'Bin ({lo_bound}–{hi_bound})', fontsize=14)
        if j == 0:
            ax.set_ylabel(br, fontsize=12)
        if i == n_rows-1:
            ax.set_xlabel('Time (s)', fontsize=12)
        if j == n_cols-1:
            ax.legend(loc='upper right', fontsize=10)
        ax.grid(False)

plt.tight_layout()
plt.show()


In [ ]:
# %% cell: Cycle-by-bin DA plot restricted to –3…+1 s, auto‑scaled Y
# (Assumes sessions, group_signals, compute_mean_ci, get_time_axis, brain_reg_to_color are in scope)

import re, bisect
import matplotlib.pyplot as plt

# 1) Build groups as before
bounds = [0, 20, 50, 100]
def cc_key_fn(session, sig_key):
    br, ch, ev = sig_key
    m = re.search(r'_cc(\d+)$', ev)
    if not m:
        return None
    X = int(m.group(1))
    bin_nr = bisect.bisect_right(bounds, X) - 1
    if bin_nr < 0 or bin_nr >= len(bounds)-1:
        return None
    cycle_nr = X % 3
    return (br, bin_nr, cycle_nr)

groups = group_signals(sessions, key_fn=cc_key_fn)
groups.pop(None, None)

# 2) Prepare time axis and mask for –3…+1 s
xs = get_time_axis()                         # originally –10…+10 s over 400 points
t0, t1 = -3, +3
mask = (xs >= t0) & (xs <= t1)
xs_sub = xs[mask]

# 3) Set up subplots, no shared Y
brain_regions = sorted({ br for (br, _, _) in groups })
bin_nrs       = list(range(len(bounds)-1))
cycle_nrs     = [0, 1, 2]

n_rows, n_cols = len(brain_regions), len(bin_nrs)
fig, axes = plt.subplots(
    n_rows, n_cols,
    figsize=(4*n_cols, 3*n_rows),
    sharex=True,    # still share X
    sharey=False    # allow each to autoscale its own Y
)

linestyles = ['-', '--', ':']

# 4) Plot mean±CI restricted to xs_sub
for i, br in enumerate(brain_regions):
    for j, b in enumerate(bin_nrs):
        ax = axes[i, j] if n_rows > 1 else axes[j]
        base_color = brain_reg_to_color.get(br, 'C0')

        for c, ls in zip(cycle_nrs, linestyles):
            key = (br, b, c)
            mat = groups.get(key)
            if mat is None:
                continue

            ys, lo, hi = compute_mean_ci(mat)
            ys, lo, hi = ys[mask], lo[mask], hi[mask]

            ax.plot(xs_sub, ys,
                    color=base_color,
                    linestyle=ls,
                    label=f'cycle {3 if c==0 else c}')
            ax.fill_between(xs_sub, lo, hi,
                            color=base_color, alpha=0.2)

        # autoscale to the data in this subplot
        ax.autoscale(enable=True, axis='y')

        # annotate axes
        lo_b, hi_b = bounds[b], bounds[b+1]
        if i == 0:
            ax.set_title(f'Bin ({lo_b}–{hi_b})', fontsize=14)
        if j == 0:
            ax.set_ylabel(br, fontsize=12)
        if i == n_rows-1:
            ax.set_xlabel('Time (s)', fontsize=12)
        if j == n_cols-1:
            ax.legend(loc='upper right', fontsize=10)
        ax.grid(False)

plt.tight_layout()
plt.show()


In [ ]:
# -3s to 1s is the most interesting timeframe

## Timeintervals
- reward collections to 1st touch
- 1st to 3rd touch
- 3rd touch to reward collection
- trial start to 1st global hit (pr. session)

## Global timestamps *
- Trial start (phot signal)
  - Delta from veh day to drug day pr. mouse


## Events
- dispimg
- 1st
- 2nd
- 3rd
- reward collection

## Drug groups
- Comparison between groups
  - Development of mouse
  - Average drug-induced changes

## Timewarping of events **

## Global AUC